In [23]:
text_file = "eng-ban.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, bangla = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((english, bangla))

In [24]:
import random
for i in range(5):
    print(random.choice(text_pairs));

('Bangladesh is a south-asian country.', '[start] বাংলাদেশ দক্ষিন এশিয়ার একটি দেশ. [end]')
('21st of February is the International Mother Language Day.', '[start] একুশে ফেব্রুয়ারি আন্তর্জাতিক মাতৃভাষা দিবস. [end]')
('Bangla is a language.', '[start] বাংলা একটি ভাষা. [end]')
('Allas! We lost the game.', '[start] হায়! আমরা খেলায় হেরেছি. [end]')
('About seventy percentage of people are literate.', '[start] বাংলাদেশের প্রায় সত্তর ভাগ মানুষ স্বাক্ষর. [end]')


In [25]:
import random
random.shuffle(text_pairs)
total_data_length = len(text_pairs)
num_val_samples = int(0.15 * total_data_length)

In [26]:
num_train_samples = (total_data_length - (2 * num_val_samples))
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples]